In [1]:
import joblib
import pickle
import torch
classifer = joblib.load("C:/Users/basma/Fourth Year/Graduation_Project/squeezenetfinetuned.pkl")


c:\Users\basma\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# loaded_model = pickle.load("squeezenetfinetuned.pkl")
import sklearn 
import numpy as np
model = pickle.load(open('C:/Users/basma/Fourth Year/Graduation_Project/squeezenetuntrained.pkl', 'rb'))
model.eval()

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [4]:
import torchvision.transforms as transforms
from PIL import Image
from PIL import ImageFile
import cv2

input_image = Image.open('C:/Users/basma/Fourth Year/Graduation_Project/SegmentedDataset/test/A040/A040_1.png').convert('RGB')

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

# Tensor of shape 1000, with confidence scores over Kemet's 1072 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)

tensor([20.2718, 11.4457, 10.5597,  ...,  5.7172,  5.0585,  2.6801])


In [5]:
# Read the categories
with open("C:/Users/basma/Fourth Year/Graduation_Project/classes_names.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

top1 = categories[top5_catid[0]]
print(top1)

A040 0.8683706521987915
A040A 0.08197950571775436
A049 0.03442128747701645
C003 0.009682654403150082
A041 0.0015775294741615653
A040


In [18]:
import matching_algorithm
matching_algorithm.dictionary_matching(top1)

[('A040', 100, 'God,King'), ('A001', 75, 'Man and his occupation'), ('A002', 75, 'Eating,speaking or Metaphorically (emotions,attitude,thinking)'), ('A035 A024', 62, 'Build, fashion(men)'), ('O001', 50, 'House')]
('A040', 100, 'God,King')


('A040', 100, 'God,King')